## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [2]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

Code to extract single tracks of IceSat-2 data

In [3]:
ATL06data = pd.read_csv('../data_prod/IceSat2ZachData',parse_dates=[4])

In [4]:
ATL06data['track'].unique()

array(['gt3r', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l'], dtype=object)

In [5]:
ATL06data.date.unique()

array(['2019-01-04T12:24:25.000000000', '2019-02-15T10:09:55.000000000',
       '2018-10-29T05:05:08.000000000', '2019-01-16T01:09:49.000000000',
       '2018-11-23T03:49:31.000000000', '2018-12-10T02:50:30.000000000',
       '2018-12-19T12:57:40.000000000', '2019-01-08T01:26:37.000000000',
       '2018-11-20T14:21:44.000000000', '2018-12-26T02:17:13.000000000',
       '2019-02-17T23:37:46.000000000', '2018-11-16T14:30:09.000000000',
       '2018-11-27T03:41:06.000000000', '2018-12-27T12:41:01.000000000',
       '2018-10-21T05:21:45.000000000', '2018-12-23T12:49:22.000000000',
       '2018-10-22T15:45:34.000000000', '2018-12-14T02:42:09.000000000',
       '2018-12-15T13:05:59.000000000', '2019-01-20T01:01:33.000000000',
       '2018-12-18T02:33:52.000000000', '2019-01-13T11:42:01.000000000',
       '2019-01-29T11:08:55.000000000', '2018-11-15T04:06:22.000000000',
       '2018-11-19T03:57:57.000000000', '2018-12-02T13:56:37.000000000',
       '2018-10-17T05:30:05.000000000', '2019-01-17

Define nearest neighbor algorithm

In [6]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Get target points from ATM csv and put into dataframe

In [7]:
ATMdata = pd.read_csv('~/xtrak/data_prod/ATMprof_20140429.csv')

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values
ATMdata.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW,geometry
0,79.150302,335.663786,415944.317386,-1.102872e+06,883.1842,18036.512689,-0.006359,-0.009239,POINT (415944.317385893 -1102872.34885821)
1,79.150190,335.665218,415976.200234,-1.102873e+06,883.0373,18068.412960,-0.005456,-0.006168,POINT (415976.2002337262 -1102873.403059846)
2,79.150078,335.666651,416008.102644,-1.102874e+06,882.8962,18100.332515,-0.003837,-0.006896,POINT (416008.1026436334 -1102874.449098021)
3,79.149967,335.668086,416040.005299,-1.102875e+06,882.7560,18132.248675,-0.004822,-0.005571,POINT (416040.0052989411 -1102875.377476926)
4,79.149855,335.669519,416071.908333,-1.102876e+06,882.5889,18164.168794,-0.003373,-0.009250,POINT (416071.9083328905 -1102876.421707155)


Make an empty dataframe for intersections
Get query points from ATL06 csv and put into dataframe
Loop through unique dates and ground tracks and find the closest point between each ground track and the flowline
Store intersection along-flow distance, z_ATM, z_ATL06 and other helpful data in Intersections dataframe

In [8]:
Intersections = {'dist_along':[],'ATM_elev':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[],'gt_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)
i = 0
for day in ATL06data.date.unique():

    for tr in ATL06data.track.unique():
        close_idx = []
        min_dist = []
        AddDatarow = []

        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
            
        
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values

        
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[j,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])
        if min(min_dist) > 1000:
            continue
        
        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        dfTran_idx = dfTran.loc[dfTran['date'] == day].index[0]
        qpt_idx = qpt_idx+dfTran_idx
        
        AddDatarow = {'dist_along':[ATMdata.loc[tpt_idx,'dist_along']],
                      'ATM_elev':[ATMdata.loc[tpt_idx,'ATM_elev']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[dfTran.loc[qpt_idx,'h']],
                      't_ATL06':[dfTran.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx],
                      'gt_ATL06':[dfTran.loc[qpt_idx,'track']]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections = Intersections.append(AddDatarow,ignore_index=True)
#         Intersections.reset_index(drop=True)
        

In [9]:
Intersections.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125289.795232,36.8164,3213.0,35.544117,2019-02-15 10:09:55,1576.0,gt1l
1,125354.749926,36.0535,3215.0,33.669106,2019-02-15 10:09:55,4931.0,gt1r
2,96208.910678,72.8016,2367.0,66.120445,2019-01-16 01:09:49,39474.0,gt3r
3,102696.987249,27.0040,2566.0,36.660946,2019-01-16 01:09:49,19999.0,gt1l
4,102632.054028,26.9541,2564.0,38.917492,2019-01-16 01:09:49,23921.0,gt1r


In [10]:
%matplotlib widget

In [11]:

f, ax = plt.subplots()
plt.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],c=Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'], c = Time,s=1)
plt.plot(Intersections['dist_along']/1000,Intersections['ATM_elev'],'.')
# cb = f.colorbar()
plt.colorbar()

# cb.ax.set_yticklabels(df.index.strftime('%b %Y'))

FigureCanvasNbAgg()

In [97]:
plt.close('all')

Save to file

In [133]:

Intersections.to_csv('~/xtrak/data_prod/Intersections_ATM20140429.csv',index=False)

In [12]:
IntersectionsSort = Intersections.sort_values(by=['t_ATL06', 'gt_ATL06'])
IntersectionsSort.reset_index()

,index,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,161,122581.170689,46.7459,3130.0,43.139470,2018-10-18 15:53:52,617767.0,gt1l
1,162,122647.058967,45.3720,3132.0,42.923240,2018-10-18 15:53:52,621074.0,gt1r
2,163,125810.129706,38.9860,3229.0,34.940983,2018-10-18 15:53:52,624012.0,gt2l
3,164,125875.304455,39.4977,3231.0,31.259686,2018-10-18 15:53:52,626687.0,gt2r
4,52,80573.944294,294.6359,1901.0,262.579130,2018-10-21 05:21:45,205271.0,gt2r
5,51,77393.293766,270.2520,1803.0,247.187770,2018-10-21 05:21:45,205803.0,gt3r
6,103,67655.902876,416.3710,1538.0,358.969240,2018-10-25 05:13:27,398282.0,gt1l
7,104,67687.240097,414.8788,1539.0,392.524100,2018-10-25 05:13:27,400497.0,gt1r
8,105,64607.360881,416.7760,1439.0,2154.631000,2018-10-25 05:13:27,401953.0,gt2l
9,106,64484.432996,423.2080,1435.0,410.355440,2018-10-25 05:13:27,403762.0,gt2r


In [16]:
InterX = IntersectionsSort.values
InterX = InterX

Dist = np.diff(InterX[:,0])
Dist[Dist == 0] = float('NaN')
InterXslopeATL06 = np.diff(InterX[:,3])/Dist
InterXslopeATM = np.diff(InterX[:,1])/Dist

# i = 0
# for i in Dist():
InterXslopeATL06[np.abs(Dist) > 100] = float('NaN')
InterXslopeATL06 = np.append(InterXslopeATL06,[0])

InterXslopeATM[np.abs(Dist) > 100] = float('NaN')
InterXslopeATM = np.append(InterXslopeATM,[float('NaN')])

print(Dist[0:10])
print(InterX.shape)
print(InterXslopeATM.shape)
print(InterXslopeATM[0:10])





[65.88827849196969 3163.0707387852744 65.17474940473039 -45301.36016124592
 -3180.6505280416604 -9737.390889608083 31.337220537185203
 -3079.87921539363 -122.92788497605216 -3104.74750294572]
(189, 7)
(189,)
[-0.020851963830978627 nan 0.007851200114669953 nan nan nan
 -0.04761749684306884 nan nan nan]


In [18]:
IntersectionsSort = IntersectionsSort.assign(slope_ATM= InterXslopeATM)

In [20]:
IntersectionsSort = IntersectionsSort.assign(slope_ATL06 = InterXslopeATL06)
IntersectionsSort.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06
161,122581.170689,46.7459,3130.0,43.139470,2018-10-18 15:53:52,617767.0,gt1l,-0.020852,-0.00328177
162,122647.058967,45.3720,3132.0,42.923240,2018-10-18 15:53:52,621074.0,gt1r,NaN,NaN
163,125810.129706,38.9860,3229.0,34.940983,2018-10-18 15:53:52,624012.0,gt2l,0.0078512,-0.0564835
164,125875.304455,39.4977,3231.0,31.259686,2018-10-18 15:53:52,626687.0,gt2r,NaN,NaN
52,80573.944294,294.6359,1901.0,262.579130,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN


In [21]:

IntersectionsSort.to_csv('~/xtrak/data_prod/Intersections_ATM20140429_wSlope.csv',index=False)